In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

file_path = 'train.csv'


df = pd.read_csv(file_path, on_bad_lines='skip')  


print(f"Размер: {df.shape}")
print(df.head())

Размер: (15436, 1)
             sample;annotation
aa;[(0     2             'O')]
aala;[(0   4             'O')]
aarcca;[(0 6             'O')]
abon;[(0   4             'O')]
abso;[(0   4       'B-BRAND')]


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15436 entries, ('aa;[(0', 2) to ('яяйца;[(0', 5)
Data columns (total 1 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   sample;annotation  15436 non-null  object
dtypes: object(1)
memory usage: 803.2+ KB


In [4]:
import ast



df = pd.read_csv(file_path, sep=';') 

print(type(df["annotation"].iloc[0]))  # str


df["annotation"] = df["annotation"].apply(ast.literal_eval)
print(type(df["annotation"].iloc[0]))
print(df["annotation"].iloc[0])

print(df.head())

<class 'str'>
<class 'list'>
[(0, 2, 'O')]
   sample         annotation
0      aa        [(0, 2, O)]
1    aala        [(0, 4, O)]
2  aarcca        [(0, 6, O)]
3    abon        [(0, 4, O)]
4    abso  [(0, 4, B-BRAND)]


In [5]:
print(df.head(50))

            sample                          annotation
0               aa                         [(0, 2, O)]
1             aala                         [(0, 4, O)]
2           aarcca                         [(0, 6, O)]
3             abon                         [(0, 4, O)]
4             abso                   [(0, 4, B-BRAND)]
5     abtoys игруш  [(0, 6, B-BRAND), (7, 12, B-TYPE)]
6    abtoys игрушк  [(0, 6, B-BRAND), (7, 13, B-TYPE)]
7   abtoys игрушки  [(0, 6, B-BRAND), (7, 14, B-TYPE)]
8            acofe                         [(0, 5, O)]
9          actimal                   [(0, 7, B-BRAND)]
10         actimel                   [(0, 7, B-BRAND)]
11       actimeuno                   [(0, 9, B-BRAND)]
12        actimino                   [(0, 8, B-BRAND)]
13         actimun                   [(0, 7, B-BRAND)]
14       actimunno                   [(0, 9, B-BRAND)]
15           activ                   [(0, 5, B-BRAND)]
16          activa                   [(0, 6, B-BRAND)]
17        

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27251 entries, 0 to 27250
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sample      27251 non-null  object
 1   annotation  27251 non-null  object
dtypes: object(2)
memory usage: 425.9+ KB


In [7]:
def to_bio(query, spans):
    labels = ["O"] * len(query.split())
    tokens = query.split()

    for start, end, tag in spans:
        word = query[start:end]
        for i, tok in enumerate(tokens):
            if word.startswith(tok):  
                if tag.startswith("B-"):
                    labels[i] = tag
                elif tag.startswith("I-"):
                    labels[i] = tag
    return list(zip(tokens, labels))

example = to_bio("абрикосы 500г global village", [(0, 8, 'B-TYPE'), (9, 13, 'B-VOLUME'), (14, 20, 'B-BRAND'), (21, 28, 'I-BRAND')])
print(example)



[('абрикосы', 'B-TYPE'), ('500г', 'B-VOLUME'), ('global', 'B-BRAND'), ('village', 'I-BRAND')]


In [8]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset

MODEL = "cointegrated/rubert-tiny2" #моделька

tokenizer = AutoTokenizer.from_pretrained(MODEL)

train_data = []
for _, row in df.iterrows():
    tokens, labels = zip(*to_bio(row["sample"], row["annotation"]))
    train_data.append({"tokens": list(tokens), "ner_tags": list(labels)})

dataset = Dataset.from_list(train_data)
print(dataset[0])


tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

C:\Users\Marsohodik\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Marsohodik\.cache\huggingface\hub\models--cointegrated--rubert-tiny2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

{'tokens': ['aa'], 'ner_tags': ['O']}


In [9]:
print(type(dataset))

<class 'datasets.arrow_dataset.Dataset'>


In [10]:
from tabulate import tabulate


first_10 = dataset.select(range(min(20, len(dataset))))

table_data = []
for i, example in enumerate(first_10):
    for token, tag in zip(example['tokens'], example['ner_tags']):
        table_data.append([i + 1, token, tag])

print(tabulate(table_data, headers=['Пример', 'Токен', 'Метка'], tablefmt='grid'))

+----------+-----------+---------+
|   Пример | Токен     | Метка   |
+==========+===========+=========+
|        1 | aa        | O       |
+----------+-----------+---------+
|        2 | aala      | O       |
+----------+-----------+---------+
|        3 | aarcca    | O       |
+----------+-----------+---------+
|        4 | abon      | O       |
+----------+-----------+---------+
|        5 | abso      | B-BRAND |
+----------+-----------+---------+
|        6 | abtoys    | B-BRAND |
+----------+-----------+---------+
|        6 | игруш     | B-TYPE  |
+----------+-----------+---------+
|        7 | abtoys    | B-BRAND |
+----------+-----------+---------+
|        7 | игрушк    | B-TYPE  |
+----------+-----------+---------+
|        8 | abtoys    | B-BRAND |
+----------+-----------+---------+
|        8 | игрушки   | B-TYPE  |
+----------+-----------+---------+
|        9 | acofe     | O       |
+----------+-----------+---------+
|       10 | actimal   | B-BRAND |
+----------+--------

In [11]:
print(dataset[28])

{'tokens': ['agama', 'треска'], 'ner_tags': ['B-BRAND', 'B-TYPE']}


In [12]:
unique_tags = set(tag for ex in train_data for tag in ex["ner_tags"])
label2id = {label: i for i, label in enumerate(sorted(unique_tags))}
id2label = {i: label for label, i in label2id.items()}

print(label2id)

{'B-BRAND': 0, 'B-PERCENT': 1, 'B-TYPE': 2, 'B-VOLUME': 3, 'I-BRAND': 4, 'I-PERCENT': 5, 'I-TYPE': 6, 'I-VOLUME': 7, 'O': 8}


In [13]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,  
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  #спецтокен
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])  
            else:
                label_ids.append(label2id[label[word_idx]])
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/27251 [00:00<?, ? examples/s]

In [14]:

tokenized_dataset

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 27251
})

In [15]:
from tabulate import tabulate


first_20 = tokenized_dataset.select(range(min(50, len(tokenized_dataset))))

table_data = []
for i, example in enumerate(first_20):

    decoded_tokens = tokenizer.convert_ids_to_tokens(example['input_ids'])

    for j, (token, label_id) in enumerate(zip(decoded_tokens, example['labels'])):
        #ID меток обратно
        if label_id == -100:
            tag_str = "IGNORE"
        else:
            tag_str = id2label.get(label_id, f"UNK_{label_id}")

        table_data.append([i + 1, j, token, label_id, tag_str])

print(tabulate(table_data, headers=['Пример', 'Позиция', 'BERT Токен', 'Label ID', 'Метка'], tablefmt='grid'))

+----------+-----------+--------------+------------+---------+
|   Пример |   Позиция | BERT Токен   |   Label ID | Метка   |
+==========+===========+==============+============+=========+
|        1 |         0 | [CLS]        |       -100 | IGNORE  |
+----------+-----------+--------------+------------+---------+
|        1 |         1 | aa           |          8 | O       |
+----------+-----------+--------------+------------+---------+
|        1 |         2 | [SEP]        |       -100 | IGNORE  |
+----------+-----------+--------------+------------+---------+
|        2 |         0 | [CLS]        |       -100 | IGNORE  |
+----------+-----------+--------------+------------+---------+
|        2 |         1 | aa           |          8 | O       |
+----------+-----------+--------------+------------+---------+
|        2 |         2 | ##la         |          8 | O       |
+----------+-----------+--------------+------------+---------+
|        2 |         3 | [SEP]        |       -100 | IG

In [16]:
from datasets import DatasetDict


dataset = tokenized_dataset.train_test_split(test_size=0.05, seed=42)


print(dataset)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 25888
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1363
    })
})


In [17]:
dataset = DatasetDict({
    "train": dataset["train"],
    "validation": dataset["test"]
})


In [18]:
print(dataset["train"])
print(dataset["validation"])

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 25888
})
Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 1363
})


In [19]:
#лишнее
dataset = dataset.remove_columns(['tokens', 'ner_tags','token_type_ids'])

In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 25888
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1363
    })
})

In [21]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(-1)

    # Игнорир спецтокенов
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_preds  = [[id2label[p] for (p, l) in zip(pred, label) if l != -100]
                   for pred, label in zip(predictions, labels)]

    precision = precision_score(true_labels, true_preds)
    recall    = recall_score(true_labels, true_preds)
    f1        = f1_score(true_labels, true_preds)

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }


In [23]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification,EarlyStoppingCallback

data_collator = DataCollatorForTokenClassification(tokenizer, padding=True)
model = AutoModelForTokenClassification.from_pretrained(
    MODEL,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

args = TrainingArguments(
    output_dir="./ner-model",
    eval_strategy="epoch",
    save_strategy="epoch",
    warmup_ratio=0.1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=2,
    remove_unused_columns=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

trainer.train()


Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Marsohodik\AppData\Local\Temp\ipykernel_34564\2444013151.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.452500,0.413143,0.869285,0.892325,0.880654


TrainOutput(global_step=1618, training_loss=0.6484211896936443, metrics={'train_runtime': 450.6566, 'train_samples_per_second': 57.445, 'train_steps_per_second': 3.59, 'total_flos': 2853618213888.0, 'train_loss': 0.6484211896936443, 'epoch': 1.0})

In [ ]:
from transformers import pipeline, AutoTokenizer

model_check = "./ner-model/checkpoint-18396"
model_name = "cointegrated/rubert-tiny2"

tokenizer = AutoTokenizer.from_pretrained(model_check)
model = AutoModelForTokenClassification.from_pretrained(model_name)

ner_pipeline = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy=None  #каждый токен отдельно
)

queries = [
    "наггетсы",
    "кисель",
    "зубочистки 100шт",
    "coca cola 1л",
    "coca cola zero",
    "форма для выпечки",
    "сок ананасовый без сахара",
    "молоко 2.5%"

]

results = []

for idx, query in enumerate(queries, 1):
    predictions = ner_pipeline(query)
    predictions = sorted(predictions, key=lambda x: x['start'])

    words = [(m.start(), m.end(), m.group()) for m in re.finditer(r'\S+', query)]

    word_labels = []
    prev_entity_type = None  #чтобы определить B/I
    for w_start, w_end, word in words:
        # ищем токены, которые полностью входят в слово
        tokens_in_word = [t for t in predictions if t['start'] >= w_start and t['end'] <= w_end]
        if not tokens_in_word:
            word_labels.append((w_start, w_end, "O"))
            prev_entity_type = None
            continue

        first_entity = tokens_in_word[0]['entity']
        base_entity = first_entity[2:] if first_entity.startswith(("B-", "I-")) else first_entity

        #для I-ENTITY,
        if prev_entity_type == base_entity:
            bio_label = f"I-{base_entity}"
        else:
            bio_label = f"B-{base_entity}"
            prev_entity_type = base_entity

        word_labels.append((w_start, w_end, bio_label))

    results.append(f"{idx};{query};{word_labels}")

print("id;search_query;annotation")
for r in results:
    print(r)

In [ ]:
file_path2 = '/content/drive/MyDrive/Colab Notebooks/submission.csv'


df2 = pd.read_csv(file_path2, sep=';')  


print(type(df2["annotation"].iloc[0]))  
  

df2["annotation"] = df2["annotation"].apply(ast.literal_eval)
print(type(df2["annotation"].iloc[0]))
print(df2["annotation"].iloc[0])

print(df2.head())

In [ ]:
true_labels = df2[['annotation']].copy()
true_labels.head()

In [ ]:
df2 = df2.drop('annotation', axis=1)
df2

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
import pandas as pd
import re


MODEL_PATH = "./ner-model/checkpoint-18396"
MODEL_NAME = "cointegrated/rubert-tiny"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForTokenClassification.from_pretrained(MODEL_PATH)

ner_pipeline = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy=None  
)


def predict_bio(query):
    predictions = ner_pipeline(query)
    predictions = sorted(predictions, key=lambda x: x['start'])

    words = [(m.start(), m.end(), m.group()) for m in re.finditer(r'\S+', query)]

    word_labels = []
    prev_entity_type = None
    for w_start, w_end, word in words:
        tokens_in_word = [t for t in predictions if t['start'] >= w_start and t['end'] <= w_end]
        if not tokens_in_word:
            word_labels.append((w_start, w_end, "O"))
            prev_entity_type = None
            continue

        first_entity = tokens_in_word[0]['entity']
        base_entity = first_entity[2:] if first_entity.startswith(("B-", "I-")) else first_entity

        # B- или I- метка
        if prev_entity_type == base_entity:
            bio_label = f"I-{base_entity}"
        else:
            bio_label = f"B-{base_entity}"
            prev_entity_type = base_entity

        word_labels.append((w_start, w_end, bio_label))

    return word_labels

#к каждому запросу
annotations = []
for idx, row in df2.iterrows():
    ann = predict_bio(row['sample'])
    annotations.append(ann)

df2['annotation'] = annotations


df2.head()


In [ ]:
from collections import defaultdict

def compute_macro_f1(df_pred, df_true, annotation_col='annotation'):
 

    y_true = df_true[annotation_col].tolist()
    y_pred = df_pred[annotation_col].tolist()

    entity_types = ['TYPE', 'BRAND', 'VOLUME', 'PERCENT']
    metrics = {etype: {'TP':0, 'FP':0, 'FN':0} for etype in entity_types}

    for true_row, pred_row in zip(y_true, y_pred):
        true_entities = defaultdict(set)
        for start, end, label in true_row:
            if label != 'O':
                etype = label.split('-')[1] 
                true_entities[etype].add((start, end))

        pred_entities = defaultdict(set)
        for start, end, label in pred_row:
            if label != 'O':
                etype = label.split('-')[1]
                pred_entities[etype].add((start, end))

        for etype in entity_types:
            TP = len(true_entities[etype] & pred_entities[etype])
            FP = len(pred_entities[etype] - true_entities[etype])
            FN = len(true_entities[etype] - pred_entities[etype])
            metrics[etype]['TP'] += TP
            metrics[etype]['FP'] += FP
            metrics[etype]['FN'] += FN

    f1_scores = {}
    for etype in entity_types:
        TP = metrics[etype]['TP']
        FP = metrics[etype]['FP']
        FN = metrics[etype]['FN']
        precision = TP / (TP + FP) if TP + FP > 0 else 0.0
        recall = TP / (TP + FN) if TP + FN > 0 else 0.0
        f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0.0
        f1_scores[etype] = f1
        print(f"{etype}: Precision={precision:.3f}, Recall={recall:.3f}, F1={f1:.3f}")

    macro_f1 = sum(f1_scores.values()) / len(entity_types)
    print(f"Macro F1: {macro_f1:.3f}")

    return f1_scores, macro_f1


In [ ]:
f1_scores, macro_f1 = compute_macro_f1(df_pred=df2, df_true=true_labels, annotation_col='annotation')


In [ ]:

df2.to_csv("/content/drive/MyDrive/Colab Notebooks/submission3.csv", sep=";", index=False)


print(df2)